Generate, load and prepare policy histories

In [31]:
import os
from datetime import datetime
import support_functions as sf


PROJECT_ROOT_DIR = "/Users/gopora/MyStuff/Dev/Workspaces/Sandbox/TroubledLife"
DATASETS_DIR = os.path.join(PROJECT_ROOT_DIR, "data")
TF_LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "tf_logs")
TRAINING_SET_DATA_FILE = "troubled_life_policy_train_data.csv"
TEST_SET_DATA_FILE = "troubled_life_policy_test_data.csv"

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
log_dir = "{}/run-{}/".format(TF_LOG_DIR, now)

sf.generate_very_simple_troubled_life_policy_data(no_of_policies=10000, runtime=5, file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
policy_histories = sf.load_troubled_life_policy_data(file_path=os.path.join(DATASETS_DIR, TRAINING_SET_DATA_FILE))
policy_histories, history_lengths, max_history_length = sf.pad_troubled_life_policy_histories(policy_histories=policy_histories)

Prepare labels and features

In [32]:
# Extract features and labels from dataset as numpy.ndarray(s)
policy_histories_features = policy_histories[['premium', 'current_capital']].as_matrix()
policy_histories_labels = policy_histories['troubled'].replace(to_replace=[False, True], value=[0, 1]).as_matrix()

# Remove column with policy ids (history_lengths is a numpy.ndarray)
policy_history_lengths =  history_lengths[:,1]

# Reshape labels from (overall_no_of_histories) to (overall_no_of_policies, maximum_history_length) and take maximum of each row (0 or 1)
policy_histories_labels = policy_histories_labels.reshape(policy_histories.index.levels[0].shape[0], -1).max(axis=1)

# Reshape features from (overall_no_of_histories, 2) to (overall_no_of_policies, maximum_history_length, 2)
policy_histories_features = policy_histories_features.reshape((policy_histories.index.levels[0].shape[0], max_history_length, -1))

Build TensorFlow graph with RNN

In [33]:
import tensorflow as tf

tf.reset_default_graph()
tf.set_random_seed(42)

# Our hyperparameters
n_steps = max_history_length
n_inputs = 2
n_neurons = 150
n_outputs = 2
learning_rate = 0.001

# Placeholders for our input sequences
seq_length = tf.placeholder(tf.int32, [None], name="seq_length")
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
y = tf.placeholder(tf.int32, [None], name="y")

# A single RNN layer of n_neurons neurons
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

# We feed the layer's output of the last timestep into a fully connected layer of 2 neurons
# (1 per class, troubled or not). Softmax layer is next
logits = tf.layers.dense(states, n_outputs)
y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

# Loss function and Adam Optimizer
loss = tf.reduce_mean(tf.cast(xentropy, tf.float32))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

accuracy_summary = tf.summary.scalar('Accuracy', accuracy)
loss_summary = tf.summary.scalar('Loss', loss)

file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [34]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

n_epochs = 100
batch_size = 200

X_batch = policy_histories_features
y_batch = policy_histories_labels
seq_length_batch = policy_history_lengths

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for i in range(policy_histories_labels.shape[0] // batch_size):
            X_batch = policy_histories_features[i * batch_size : (i + 1) * batch_size]
            y_batch = policy_histories_labels[i * batch_size : (i + 1) * batch_size]
            seq_length_batch = policy_history_lengths[i * batch_size : (i + 1) * batch_size]
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})

        y_batch_pred = y_pred.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        #acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})

        print(epoch, "Train accuracy:", acc_train)#, "Test accuracy:", acc_test)
        print(loss.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch}))
        print(confusion_matrix(y_batch, y_batch_pred))
        print(classification_report(y_batch, y_batch_pred, target_names=['ok', 'troubled']))

        loss_summary_str = loss_summary.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        accuracy_summary_str = accuracy_summary.eval(feed_dict={X: X_batch, y: y_batch, seq_length: seq_length_batch})
        file_writer.add_summary(loss_summary_str, epoch)
        file_writer.add_summary(accuracy_summary_str, epoch)

file_writer.close()

0 Train accuracy: 0.85
4.14799
[[153   0]
 [ 30  17]]
             precision    recall  f1-score   support

         ok       0.84      1.00      0.91       153
   troubled       1.00      0.36      0.53        47

avg / total       0.87      0.85      0.82       200



1 Train accuracy: 0.86
4.82602
[[153   0]
 [ 28  19]]
             precision    recall  f1-score   support

         ok       0.85      1.00      0.92       153
   troubled       1.00      0.40      0.58        47

avg / total       0.88      0.86      0.84       200



2 Train accuracy: 0.835
8.57811
[[153   0]
 [ 33  14]]
             precision    recall  f1-score   support

         ok       0.82      1.00      0.90       153
   troubled       1.00      0.30      0.46        47

avg / total       0.86      0.83      0.80       200



3 Train accuracy: 0.85
7.82198
[[153   0]
 [ 30  17]]
             precision    recall  f1-score   support

         ok       0.84      1.00      0.91       153
   troubled       1.00      0.36      0.53        47

avg / total       0.87      0.85      0.82       200



4 Train accuracy: 0.955
0.937053
[[153   0]
 [  9  38]]
             precision    recall  f1-score   support

         ok       0.94      1.00      0.97       153
   troubled       1.00      0.81      0.89        47

avg / total       0.96      0.95      0.95       200



...

95 Train accuracy: 1.0
0.0
[[153   0]
 [  0  47]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       153
   troubled       1.00      1.00      1.00        47

avg / total       1.00      1.00      1.00       200



96 Train accuracy: 1.0
0.0
[[153   0]
 [  0  47]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       153
   troubled       1.00      1.00      1.00        47

avg / total       1.00      1.00      1.00       200



97 Train accuracy: 1.0
0.0
[[153   0]
 [  0  47]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       153
   troubled       1.00      1.00      1.00        47

avg / total       1.00      1.00      1.00       200



98 Train accuracy: 1.0
0.0
[[153   0]
 [  0  47]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       153
   troubled       1.00      1.00      1.00        47

avg / total       1.00      1.00      1.00       200



99 Train accuracy: 1.0
0.0
[[153   0]
 [  0  47]]
             precision    recall  f1-score   support

         ok       1.00      1.00      1.00       153
   troubled       1.00      1.00      1.00        47

avg / total       1.00      1.00      1.00       200

